# Optimal winch control for kites with mass

TODO:
- Use pd dataframe to save all the data. Then it can have higher/lower
  resolution at points of interest.
- yaml file for MegAWES parameters.
- Use Cl/Cd_kite+tether for Dylan's paper and set tether.CD to zero in
  quasi-steady model? Then we do use more 'realistic' data.
- Convert string paths to os.path paths.

Further optimizations for the winch controller:
- Could do beta_fixed and phi_fixed as function of L.

In [1]:
# When working with submodules it is a bit awkward to import them, especially if
# they're not python packages (with __init__.py files).
# https://stackoverflow.com/questions/29746958/how-to-import-python-file-from-git-submodule
import sys
import os

(parent_folder_path, _) = os.path.split(os.getcwd())
workshop_path = os.path.join(parent_folder_path, "workshop")
sys.path.append(workshop_path)

import qsm

In [2]:
# Import required packages.
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import yaml
from itertools import product
from matplotlib import cm
from plotly.subplots import make_subplots

# Configure the default plotting settings
size=13
params = {'legend.fontsize': 'large',
          'figure.figsize': (8,5),
          'axes.labelsize': size,
          'axes.titlesize': size,
          'xtick.labelsize': size*0.85,
          'ytick.labelsize': size*0.85,
          'axes.titlepad': 25}
plt.rcParams.update(params)

In [7]:
# Load the parameter file. These are values that will remain fixed.
with open("../parameters/MegAWES.yaml", "r") as f:
    try:
        params = yaml.safe_load(f)
        kite = params['kite']
        environment = params['environment']

    except yaml.YAMLError as exc:
        print(exc)

In [8]:
# Is yaml what we want here? How adaptive is it really?
# How to set ranges?
# Where to store these ranges (DataFrame?) First try to retrieve existing
#  DataFrame and add rows with new values. If none is existing, first make a new
#  one (empty?) and then add rows with new values.
# All combinations of two lists: itertools.product() or pd.cartesian.product().
# Calculate answer afterwards? What to put if it doesn't converge vs what to put
#  if the row is empty (hasn't been calculated yet).
# Store all calculated values. From this we can optimize the traction phase, or
#  traction and retraction together.

In [41]:
# Load all existing data or start with an empty DataFrame if there is none.
# TODO: fname = "quasi_steady_states_test"
if os.path.isfile("../results/quasi_steady_states_test.csv"):
    quasi_steady_states = pd.read_csv("../results/quasi_steady_states_test.csv", index_col=0)
else:
    quasi_steady_states = pd.DataFrame()

In [42]:
quasi_steady_states

,windspeed_mps,phi_deg,P,f,F
0,5.0,0.0,1.0,0.3,1000.0
1,5.0,45.0,1.0,0.3,1000.0
2,30.0,0.0,1.0,0.3,1000.0
3,30.0,45.0,1.0,0.3,1000.0


In [120]:
# Make arrays of values where we want to have the steady state calculated for.
# If this value already exists in the `quasi_steady_states` DataFrame the
# calculation will not be done again.
ranges = {
    "vw_mps": np.linspace(5, 30, 3),   # windspeed [m/s]
    "phi_deg": np.linspace(0, 45, 3),  # azimuth [deg]
    "Ftg_N": np.logspace(0, 8, 2),     # tether force ground [N]
}

# Combine these new values into the existing dataframe, using the values from
# the existing dataframe where possible.
df_new = pd.DataFrame(product(*ranges.values()), columns=ranges.keys())
df_new

,windspeed_mps,phi_deg
0,5.0,0.0
1,5.0,22.5
2,5.0,45.0
3,17.5,0.0
4,17.5,22.5
5,17.5,45.0
6,30.0,0.0
7,30.0,22.5
8,30.0,45.0


In [121]:
quasi_steady_states = quasi_steady_states.merge(df_new, how='outer', on=list(ranges.keys()))
quasi_steady_states

,windspeed_mps,phi_deg,P,f,F
0,5.0,0.0,1.0,0.3,1000.0
1,5.0,45.0,1.0,0.3,1000.0
2,30.0,0.0,1.0,0.3,1000.0
3,30.0,45.0,1.0,0.3,1000.0
4,17.5,0.0,1.0,0.3,1000.0
5,17.5,45.0,1.0,0.3,1000.0
6,5.0,22.5,NaN,NaN,NaN
7,17.5,22.5,NaN,NaN,NaN
8,30.0,22.5,NaN,NaN,NaN


In [122]:
def ss_from_pd_row(row: pd.Series):

    P = 1
    f = 0.3
    F = 1e3
    return P, f, F


In [123]:
def ss_from_pd_row_to_pd_series(row: pd.Series):

    P = row["windspeed_mps"]
    f = 0.3
    F = 1e3
    return pd.Series([P, f, F], index=['P', 'f', 'F'])

In [124]:
df_PfF = quasi_steady_states[quasi_steady_states.isna().any(axis=1)].apply(ss_from_pd_row_to_pd_series, axis=1)
df_PfF

,P,f,F
6,5.0,0.3,1000.0
7,17.5,0.3,1000.0
8,30.0,0.3,1000.0


In [125]:
quasi_steady_states.update(df_PfF)  # Nice that works.

In [126]:
quasi_steady_states

,windspeed_mps,phi_deg,P,f,F
0,5.0,0.0,1.0,0.3,1000.0
1,5.0,45.0,1.0,0.3,1000.0
2,30.0,0.0,1.0,0.3,1000.0
3,30.0,45.0,1.0,0.3,1000.0
4,17.5,0.0,1.0,0.3,1000.0
5,17.5,45.0,1.0,0.3,1000.0
6,5.0,22.5,5.0,0.3,1000.0
7,17.5,22.5,17.5,0.3,1000.0
8,30.0,22.5,30.0,0.3,1000.0


In [112]:
# # If a row has a NAN value, we haven't calculated the quasi-steady state for it yet.
# PfF = quasi_steady_states[quasi_steady_states.isna().any(axis=1)].apply(ss_from_pd_row, axis=1).apply(pd.Series)
# quasi_steady_states[quasi_steady_states.isna().any(axis=1)].loc([["P", "f", "F"]]) = PfF
# # quasi_steady_states[["P", "f", "F"]] = quasi_steady_states.apply(ss_from_pd_row, axis=1).apply(pd.Series)
# # quasi_steady_states

SyntaxError: cannot assign to function call here. Maybe you meant '==' instead of '='? (2374174135.py, line 3)

In [ ]:
# # That all didn't really work. I think it's better anyway to just loop over the rows.
# for (i, row) in quasi_steady_states.iterrows():
    

In [40]:
quasi_steady_states.to_csv("../results/quasi_steady_states_test.csv")